In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import Dict, Any


# Define prompt templates
nurse_prompt = ChatPromptTemplate.from_template(
    "Role: ER Triage Nurse\nInstructions: Determine ESI (1-5) with reasoning...\nPatient Note: {note}\nPrevious Doctor Input: {doctor_msg}\nYour Response:"
)
doctor_prompt = ChatPromptTemplate.from_template(
    "Role: ER Doctor\nInstructions: List possible diagnoses and comment on priority...\nPatient Note: {note}\nNurse's Assessment: {nurse_msg}\nYour Response:"
)



In [19]:
import os
from dotenv import load_dotenv

load_dotenv()

if 'OPENAI_API_KEY' not in os.environ:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

In [20]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0)

In [22]:
def nurse_step(state: Dict[str, Any]) -> Dict[str, Any]:
    """Process nurse's assessment with state preservation"""
    inputs = {
        "note": state["note"],
        "doctor_msg": state.get("doctor_msg", "")
    }
    response = (nurse_prompt | llm).invoke(inputs)
    return {**state, "nurse_msg": response.content}  # Preserve existing state

def doctor_step(state: Dict[str, Any]) -> Dict[str, Any]:
    """Process doctor's analysis with state preservation"""
    inputs = {
        "note": state["note"],
        "nurse_msg": state["nurse_msg"]
    }
    response = (doctor_prompt | llm).invoke(inputs)
    return {
        **state,  # Preserve existing state
        "doctor_msg": response.content,
        "iteration": state.get("iteration", 0) + 1
    }

# Build workflow
workflow = StateGraph(state_schema=dict)
workflow.add_node("Nurse", nurse_step)
workflow.add_node("Doctor", doctor_step)
workflow.set_entry_point("Nurse")
workflow.add_edge("Nurse", "Doctor")
workflow.add_conditional_edges(
    "Doctor",
    lambda state: "Nurse" if state.get("iteration", 0) < 2 else END
)

app = workflow.compile()

# Test run
patient_note = "45yo male presenting with chest pain radiating to left arm..."
result = app.invoke({"note": patient_note})

print("Final Assessment:", result["nurse_msg"])
print("\nFinal Analysis:", result["doctor_msg"])

Final Assessment: Based on the provided patient note and the previous doctor's input, I would assign an Emergency Severity Index (ESI) level of **2** to this patient.

**Reasoning:**

1. **High Risk of Deterioration:** The patient is a 45-year-old male presenting with chest pain radiating to the left arm, which is a classic symptom of myocardial infarction (MI) or unstable angina. These conditions are life-threatening and require immediate evaluation and intervention.

2. **Need for Urgent Evaluation:** The symptoms suggest a high likelihood of acute coronary syndrome, necessitating urgent diagnostic tests such as an ECG and cardiac biomarkers. The potential for rapid deterioration into a more critical state (e.g., full-blown MI) underscores the need for prompt assessment.

3. **Immediate Interventions Required:** The patient will likely need continuous monitoring, oxygen administration, IV access, and possibly medications like nitroglycerin or aspirin while awaiting further evaluation

In [23]:
# Add validation checks
assert "note" in result, "Patient note missing in final state"
assert result["iteration"] == 2, f"Unexpected iterations: {result['iteration']}"
assert "chest pain" in result["nurse_msg"], "Key symptom missing in nurse analysis"
assert "Acute Coronary Syndrome" in result["doctor_msg"], "Key diagnosis missing"

In [26]:
critical_note = "30yo female unresponsive with BP 70/40..."
critical_result = app.invoke({"note": critical_note})

In [27]:
critical_result["nurse_msg"]

'Based on the provided information, the patient is a 30-year-old female who is unresponsive and has a blood pressure of 70/40 mmHg. This presentation is concerning for a life-threatening condition, and immediate intervention is required.\n\n**Emergency Severity Index (ESI) Level: 1**\n\n**Reasoning:**\n- **Unresponsiveness**: The patient is unresponsive, indicating a potential compromise in the airway, breathing, or circulation. This is a critical sign that necessitates immediate medical attention.\n- **Hypotension (BP 70/40)**: The extremely low blood pressure suggests a state of shock, which can be due to several life-threatening conditions such as septic shock, hypovolemic shock, or cardiogenic shock. Each of these conditions requires urgent intervention to prevent further deterioration and potential death.\n- **Need for Immediate Intervention**: The patient requires rapid assessment and treatment, including airway management, fluid resuscitation, and possibly medications (e.g., ant